<a href="https://colab.research.google.com/github/shouvikcirca/LLMs/blob/main/langchain2_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! python --version
! pip install --upgrade langchain
! pip install python-dotenv
! pip install --upgrade openai
! pip install pypdf
! pip install tiktoken
! pip install chromadb
! pip install cohere


In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# ! ls drive/MyDrive

In [5]:
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv, find_dotenv
import os
import sys
import openai

In [6]:
a = load_dotenv('drive/MyDrive/env')

In [13]:
loader = PyPDFLoader('drive/MyDrive/C-CARES_FRS_Grievance_Redressal.pdf')
pages = loader.load()
len(pages)

17

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, TokenTextSplitter

In [15]:
chunk_size = 50
chunk_overlap = 10

In [16]:
t_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [17]:
docs = t_splitter.split_documents(pages)
len(docs)

165

In [18]:
from langchain.embeddings import CohereEmbeddings
embedding = CohereEmbeddings(
    model="embed-english-light-v3.0", cohere_api_key=os.environ['COHERE_API_KEY']
)

In [10]:
from langchain.vectorstores import Chroma

In [11]:
persist_directory = 'docs/chroma/'

In [19]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

In [7]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [3]:
from langchain.llms import Cohere

In [9]:
llm = Cohere(cohere_api_key = os.environ['COHERE_API_KEY'], temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [23]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [21]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [24]:
question = "What is the grievance redressal module designed for ?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

The Grievance Redressal Module is designed to operate digitally as a 3 -tier hierarchy of the Grievance Management process at CMPFO. The grievance is raised by the member and submitted to the portal. 

Is there anything else I can help you extract from this text?
----------------------------------------------------------------------------------------------------
Document 2:

The Grievance Redressal Module takes care of the lifetime of the grievance starting from Online Submission by the member till the Resolution of the grievance.

Is there anything else I can help you extract from this text?
----------------------------------------------------------------------------------------------------
Document 3:

10.1 Description  
The Grievance Redressal Module is designed

Is there anything else I can help you extract from this text?
----------------------------------------------------------------------------------------------------
Document 4:

Officer of a particular RO or CC .